In [ ]:
import holoviews as hv
import pandas as pd
import numpy as np
import panel as pn
import selenium as sm
pn.extension("tabulator")

In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import itertools as it
import pmdarima as pm

In [ ]:
import mysql.connector as cn

##### ML libs

In [ ]:
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout
from sklearn import metrics, linear_model
from scalecast.auxmodels import auto_arima

## 1. Data acquisition 
using MySQL queries from prestashop database into Pandas data frames

In [ ]:

# if 'data' not in pn.state.cache.keys():
try:
    mydb = cn.connect(host="localhost", database='prestashop',
                      user="admin", passwd="admin", use_pure=True)

    query_orders = "SELECT \
                        od.product_id, \
                        od.product_ean13 AS EAN, \
                        od.product_quantity AS 'Wielkosc_zamowienia', \
                        od.total_price_tax_incl AS 'Calkowita_wartosc_zamowienia', \
                        o.date_add AS 'Data', o.id_order  , pl.name\
                        FROM ps_orders o \
                        INNER JOIN ps_order_detail od \
                            ON o.id_order = od.id_order\
                        INNER JOIN ps_product_lang pl \
                            ON od.product_id = pl.id_product;"

    query_sale = "SELECT \
                    pl.name, \
                    ROUND(SUM(od.product_quantity)) AS Rozchod ,\
                    ROUND(SUM(od.product_quantity * od.product_price), 2) AS 'Calk_przych',\
                    ROUND(SUM(od.product_quantity * p.wholesale_price), 2) AS 'Calk_koszt', \
                    ROUND(SUM(od.product_quantity * od.product_price)- SUM(od.product_quantity * p.wholesale_price), 2) AS 'Calk_zysk',\
                    sa.quantity AS Stan, \
                    ROUND(p.wholesale_price*sa.quantity,2) AS 'Obecna_wart_zapasu', \
                    ROUND(((p.price - p.wholesale_price) / p.price ) * 100, 2) AS 'Marza' ,\
                    ROUND((SUM(od.product_quantity * p.wholesale_price)) / ((sa.physical_quantity * p.wholesale_price) / 8),2) AS 'Wsk_rotacji',\
                    ROUND(SUM(od.product_quantity)/12,2) AS 'Wsk_obrotu_miesieczny' ,\
                    ROUND(sa.quantity*365/SUM(od.product_quantity),2) AS 'Wsk_wystarczalnosci_roczny' ,\
                    ROUND(((p.price - p.wholesale_price) / p.wholesale_price ) * 100, 2) AS 'Narzut',\
                    ROUND(((sa.physical_quantity * od.product_price)+(sa.quantity * od.product_price)) / 2, 2) AS 'Sr_wart_zapasow',\
                    p.ean13 AS EAN \
                    \
                    FROM ps_product p \
                    INNER JOIN ps_order_detail od \
                        ON p.id_product = od.product_id \
                    INNER JOIN ps_stock_available sa \
                        ON p.id_product = sa.id_product \
                    INNER JOIN ps_product_lang pl \
                        ON p.id_product = pl.id_product \
                    GROUP BY p.id_product, sa.physical_quantity, sa.quantity, od.product_price, pl.name, 'Marza',p.wholesale_price;"

    query_warehouse = "SELECT \
                        ROUND(SUM(p.wholesale_price * sa.quantity),2) AS Obecna_wartosc_magazynu,\
                        sum(sa.quantity) AS 'stan_magazynowy' \
                        \
                        FROM ps_product p \
                        INNER JOIN ps_stock_available sa \
                            ON p.id_product = sa.id_product;"
    query_sale_year = "SELECT \
                        YEAR(o.date_add) AS Rok, \
                        pl.name, \
                        ROUND(SUM(od.product_quantity)) AS Rozchod ,\
                        ROUND(SUM(od.product_quantity * od.product_price), 2) AS 'Calk_przych',\
                        ROUND(SUM(od.product_quantity * p.wholesale_price), 2) AS 'Calk_koszt', \
                        ROUND(SUM(od.product_quantity * od.product_price)- SUM(od.product_quantity * p.wholesale_price), 2) AS 'Calk_zysk',\
                        sa.quantity AS Stan, \
                        ROUND(p.wholesale_price*sa.quantity,2) AS 'Obecna_wart_zapasu', \
                        ROUND(((p.price - p.wholesale_price) / p.price ) * 100, 2) AS 'Marza' , \
                        ROUND((SUM(od.product_quantity * od.product_price)) / ((sa.physical_quantity * od.product_price)/ 8),2) AS 'Wsk_rotacji',\
                        ROUND(SUM(od.product_quantity)/12,2) AS 'Wsk_obrotu_miesieczny' ,\
                        ROUND(sa.quantity*365/SUM(od.product_quantity),2) AS 'Wsk_wystarczalnosci_roczny' ,\
                        ROUND(((p.price - p.wholesale_price) / p.wholesale_price ) * 100, 2) AS 'Narzut',\
                        ROUND(((sa.physical_quantity * od.product_price)+(sa.quantity * od.product_price)) / 2, 2) AS 'Sr_wart_zapasow',\
                        p.ean13 AS EAN \
                        \
                        FROM ps_product p \
                        INNER JOIN ps_order_detail od \
                            ON p.id_product = od.product_id \
                        INNER JOIN ps_stock_available sa \
                            ON p.id_product = sa.id_product \
                        INNER JOIN ps_product_lang pl \
                            ON p.id_product = pl.id_product \
                        INNER JOIN ps_orders o \
                            ON o.id_order = od.id_order \
                        GROUP BY YEAR(o.date_add), p.id_product, sa.physical_quantity, sa.quantity, od.product_price, pl.name, 'Marza',p.wholesale_price;"

    # sql queries to pandas df
    df_orders = pd.read_sql(query_orders, mydb)
    df_sale = pd.read_sql(query_sale, mydb)
    df_warehouse = pd.read_sql(query_warehouse, mydb)
    df_sale_year = pd.read_sql(query_sale_year, mydb)
    df_sale_year = df_sale_year[df_sale_year['Rok']
                                < 2023]  # excluded one day from 2023 year

    mydb.close()  # close the connection

except Exception as e:
    mydb.close()
    print(str(e))

# else:

#    df = pn.state.cache['data']

## 2. Export/Import of data to/from .csv

In [ ]:
# save to .csv
df_warehouse.to_csv("dane_dashboard/df_warehouse.csv")
df_sale.to_csv("dane_dashboard/df_sale.csv")
df_sale_year.to_csv("dane_dashboard/df_sale_year.csv")
df_orders.to_csv("dane_dashboard/df_orders.csv")

# import .csv files to dataframes
# df_orders = pd.read_csv("df_orders.csv")
# df_sale = pd.read_csv("df_sale.csv")
# df_warehouse = pd.read_csv("df_warehouse.csv")
# df_sale_year = pd.read_csv("df_sale_year.csv")

# 3. Controlers

### Years slider

In [ ]:
# Define Panel widgets
year_select = pn.widgets.Select(
    name='Wybierz rok:', options=[2019, 2020, 2021, 2022])

### Select from list widget
select one product by name 

In [ ]:
name = list(df_sale.name.unique())

select = pn.widgets.Select(
    name='Wybierz produkt po nazwie', options=name, height=50)

# 4. Widgets

#### Title with name and EAN

In [ ]:
def title_ean(name_2):
    prod_name = df_sale[df_sale['name'] == name_2].name.values[0]
    ean_number = df_sale[df_sale['name'] == name_2].EAN.values[0]
    # return print('Nazwa: ',prod_name, '| EAN:', ean_number)
    return pn.pane.Markdown(f""" # {prod_name} | EAN: _{ean_number}_""", width=900)


header_title_ean = pn.interact(title_ean, name_2=select)
header_title_ean = header_title_ean[1]

##### Size of number widgets

In [ ]:
fontSize = '32pt'
titleSize = '15pt'

### Sale indicators warehouse

In [ ]:
# Całkowity przychód ze sprzedaży
def total_income(year_given):
    x = round(df_sale_year[(df_sale_year['Rok'] ==
              year_given)].Calk_przych.sum(), 2)

    output = pn.indicators.Number(
        name='Całkowity przychód', value=x, format='{value} zł',
        # colors=[(df_sale['Calk_przych'].median(), 'red'), (df_sale['Calk_przych'].mean(
        # ), 'gold'), (df_sale['Calk_przych'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize
    )
    return output


number_total_income = pn.interact(total_income, year_given=year_select)
number_total_income = number_total_income[1]


# Całkowity zysk ze sprzedaży
def total_profit(year_given):
    x = round(df_sale_year[(df_sale_year['Rok']
              == year_given)].Calk_zysk.sum(), 2)

    output = pn.indicators.Number(
        name='Całkowity zysk', value=x, format='{value} zł',
        # colors=[(df_sale['Calk_zysk'].median(), 'red'), (df_sale['Calk_zysk'].mean(
        # ), 'gold'), (df_sale['Calk_zysk'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize
    )
    return output


number_total_profit = pn.interact(
    total_profit, year_given=year_select)
number_total_profit = number_total_profit[1]
number_total_profit

# Średnia roczna marża


def mean_total_margin(year_given):
    x = round(
        df_sale_year[(df_sale_year['Rok'] == year_given)].Marza.mean(), 2)

    output = pn.indicators.Number(
        name='Średnia marża', value=x, format='{value} %',
        # colors=[(df_sale['Marza'].median(), 'red'), (df_sale['Marza'].mean(
        # ), 'gold'), (df_sale['Marza'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize
    )
    return output


number_mean_total_margin = pn.interact(
    mean_total_margin, year_given=year_select)
number_mean_total_margin = number_mean_total_margin[1]
number_mean_total_margin

# Średni roczny narzut


def mean_total_markup(year_given):
    x = round(
        df_sale_year[(df_sale_year['Rok'] == year_given)].Narzut.mean(), 2)

    output = pn.indicators.Number(
        name='Średni narzut', value=x, format='{value} %',
        # colors=[(df_sale['Narzut'].median(), 'red'), (df_sale['Narzut'].mean(
        # ), 'gold'), (df_sale['Narzut'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize
    )
    return output


number_mean_total_markup = pn.interact(
    mean_total_markup, year_given=year_select)
number_mean_total_markup = number_mean_total_markup[1]
number_mean_total_markup

In [ ]:
df_sale_year['Wsk_rotacji'].mean()

### Logistic indicators warehouse

In [ ]:
# Wartość towarów w magazynie
Total_warehouse_value = round(df_warehouse.iat[0, 0])
number_Total_warehouse_value = pn.indicators.Number(
    name='Obecna wartość zapasów', value=Total_warehouse_value, format='{value} zł',
    colors=[(33, 'green'), (66, 'gold'), (100, 'red')],
    font_size=fontSize,
    title_size=titleSize
)

# Wskaźnik wystarczalności magazynowej
warehouse_sufficiency_rate = round(
    (df_warehouse.iat[0, 1] * 365) / df_orders['Wielkosc_zamowienia'].sum())
number_warehouse_sufficiency_rate = pn.indicators.Number(
    name='Obecny wskaźnik wystarczalności magazynowej', value=warehouse_sufficiency_rate, format='{value} dni',
    colors=[(33, 'green'), (66, 'gold'), (100, 'red')],
    font_size=fontSize,
    title_size=titleSize
)

# Średni wskaźnik rotacji w magazynie


def mean_warehouse_rotation(year_given):
    x = round(df_sale_year[(df_sale_year['Rok'] ==
              year_given)].Wsk_rotacji.mean(), 2)

    output = pn.indicators.Number(
        name='Średni wskaźnik rotacji', value=x, format='{value}',
        colors=[(df_sale_year['Wsk_rotacji'].median(), 'red'),
                (df_sale_year['Wsk_rotacji'].mean(), 'gold'),
                (df_sale_year['Wsk_rotacji'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize,
        width=220
    )
    return output


number_mean_rotation = pn.interact(
    mean_warehouse_rotation, year_given=year_select)
number_mean_rotation = number_mean_rotation[1]


def warehouse_turnover_rate(year_given):
    x = round(
        df_sale_year[(df_sale_year['Rok'] == year_given)].Wsk_obrotu_miesieczny.mean(), 2)

    output = pn.indicators.Number(
        name='Średni miesięczny wsk. obrotu magazynowego', value=x, format='{value}',
        # colors=[(df_sale['Narzut'].median(), 'red'), (df_sale['Narzut'].mean(
        # ), 'gold'), (df_sale['Narzut'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize,
        width=220
    )
    return output


number_warehouse_turnover_rate = pn.interact(
    warehouse_turnover_rate, year_given=year_select)
number_warehouse_turnover_rate = number_warehouse_turnover_rate[1]

### Sale indicators for specific product

In [ ]:
# Przychód ze sprzedaży produktu
def product_income(name_2, year_given):
    x = round(df_sale_year[(df_sale_year['name'] == name_2) &
                           (df_sale_year['Rok'] == year_given)].Calk_przych.values[0], 2)

    output = pn.indicators.Number(
        name='Przychód', value=x, format='{value} zł',
        colors=[(df_sale['Calk_przych'].median(), 'red'), (df_sale['Calk_przych'].mean(
        ), 'gold'), (df_sale['Calk_przych'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize
    )
    return output


number_product_income = pn.interact(
    product_income, name_2=select, year_given=year_select)
number_product_income = number_product_income[1]


# Zysk ze sprzedaży produktu
def product_profit(name_2, year_given):
    x = round(df_sale_year[(df_sale_year['name'] == name_2) &
                           (df_sale_year['Rok'] == year_given)].Calk_zysk.values[0], 2)

    output = pn.indicators.Number(
        name='Zysk', value=x, format='{value} zł',
        colors=[(df_sale['Calk_zysk'].median(), 'red'), (df_sale['Calk_zysk'].mean(
        ), 'gold'), (df_sale['Calk_zysk'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize
    )
    return output


number_profit = pn.interact(
    product_profit, name_2=select, year_given=year_select)
number_profit = number_profit[1]


# Udział produktu w sprzedaży
def sale_share(name_2, year_given):
    x = round(df_sale_year[(df_sale_year['name'] == name_2) &
                           (df_sale_year['Rok'] == year_given)].Calk_przych.values[0] /
              df_sale_year[(df_sale_year['Rok'] == year_given)].Calk_przych.sum() * 100, 2)

    output = pn.indicators.Number(
        name='Udział w sprzedaży', value=x, format='{value} %',
        colors=[(df_sale['Narzut'].median(), 'red'), (df_sale['Narzut'].mean(
        ), 'gold'), (df_sale['Narzut'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize
    )
    return output


number_sale_share = pn.interact(
    sale_share, name_2=select, year_given=year_select)
number_sale_share = number_sale_share[1]


# Marża na sprzedaży produktu
def product_margin(name_2, year_given):
    x = round(df_sale_year[(df_sale_year['name'] == name_2) &
                           (df_sale_year['Rok'] == year_given)].Marza.values[0], 2)

    output = pn.indicators.Number(
        name='Marża', value=x, format='{value} %',
        colors=[(df_sale['Marza'].median(), 'red'), (df_sale['Marza'].mean(
        ), 'gold'), (df_sale['Marza'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize
    )
    return output


number_margin = pn.interact(
    product_margin, name_2=select, year_given=year_select)
number_margin = number_margin[1]

### Logistic indicators for specific product

In [ ]:
# Wartość towarów w magazynie/ Wskaźnik zapasu
def product_stock_value(name_2, year_given):
    x = round(df_sale_year[(df_sale_year['name'] == name_2) &
                           (df_sale_year['Rok'] == year_given)].Obecna_wart_zapasu.values[0])

    output = pn.indicators.Number(
        name='Obecna wartość zapasu', value=x, format='{value} zł',
        colors=[(df_sale['Obecna_wart_zapasu'].median(), 'red'),
                (df_sale['Obecna_wart_zapasu'].mean(), 'gold'),
                (df_sale['Obecna_wart_zapasu'].max(), 'Green')],
        font_size='19pt',
        title_size='17pt',
        width=160,
        # height = 110
    )
    return output


number_stock_value = pn.interact(
    product_stock_value, name_2=select, year_given=year_select)
number_stock_value = number_stock_value[1]

# Ilość towaru w magazynie


def product_stock(name_2, year_given):
    x = round(df_sale_year[(df_sale_year['name'] == name_2) &
                           (df_sale_year['Rok'] == year_given)].Stan.values[0])

    output = pn.indicators.Number(
        name='Ilość sztuk zapasu', value=x, format='{value} szt.',
        colors=[(df_sale['Stan'].median(), 'red'),
                (df_sale['Stan'].mean(), 'gold'),
                (df_sale['Stan'].max(), 'Green')],
        font_size='19pt',
        title_size='17pt',
        width=130,
        # height = 110
    )
    return output


number_product_stock = pn.interact(
    product_stock, name_2=select, year_given=year_select)
number_product_stock = number_product_stock[1]


# Wskaźnik wystarczalności magazynowej - Na tyle dni wystarczy towaru, na podstawie historycznej sprzedaży
def product_sufficiency_rate(name_2, year_given):
    x = df_sale_year[(df_sale_year['name'] == name_2) &
                     (df_sale_year['Rok'] == year_given)].Wsk_wystarczalnosci_roczny.values[0]
    output = pn.indicators.Number(
        name='Wskaźnik wystarczalności magazynowej', value=x, format='{value} dni',
        colors=[(df_sale['Wsk_wystarczalnosci_roczny'].median(), 'red'),
                (df_sale['Wsk_wystarczalnosci_roczny'].mean(), 'gold'),
                (df_sale['Wsk_wystarczalnosci_roczny'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize,
        width=250

    )
    return output


number_product_sufficiency_rate = pn.interact(
    product_sufficiency_rate, name_2=select, year_given=year_select)
number_product_sufficiency_rate = number_product_sufficiency_rate[1]


# Wskaźnik rotacji magazynowej
def product_rotation(name_2, year_given):
    x = round(df_sale_year[(df_sale_year['name'] == name_2) &
                           (df_sale_year['Rok'] == year_given)].Wsk_rotacji.values[0], 2)
    output = pn.indicators.Number(
        name='Wskaźnik rotacji magazynowej', value=x, format='{value}',
        colors=[(df_sale_year['Wsk_rotacji'].median(), 'red'),
                (df_sale_year['Wsk_rotacji'].mean(), 'gold'),
                (df_sale_year['Wsk_rotacji'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize,
        width=250
    )
    return output


number_product_rotation = pn.interact(
    product_rotation, name_2=select, year_given=year_select)
number_product_rotation = number_product_rotation[1]

# Wskaźnik obrotu magazynowego


def product_turnover_rate(name_2, year_given):
    x = round(df_sale_year[(df_sale_year['name'] == name_2) &
                           (df_sale_year['Rok'] == year_given)].Wsk_obrotu_miesieczny.values[0], 2)

    output = pn.indicators.Number(
        name='Miesięczny wskaźnik obrotu magazynowego', value=x, format='{value}',
        colors=[(df_sale['Wsk_obrotu_miesieczny'].median(), 'red'),
                (df_sale['Wsk_obrotu_miesieczny'].mean(), 'gold'),
                (df_sale['Wsk_obrotu_miesieczny'].max(), 'Green')],
        font_size=fontSize,
        title_size=titleSize,
        width=250,
        height=120
    )
    return output


number_turnover_rate = pn.interact(
    product_turnover_rate, name_2=select, year_given=year_select)
number_turnover_rate = number_turnover_rate[1]

# 5. Plots

### Data preprocess

In [ ]:
# data frame with  months of the four years for each product

df_month_orders = df_orders[["Wielkosc_zamowienia", "name"]]
df_month_orders['miesiac'] = pd.to_datetime(df_orders.Data).dt.month
df_month_orders['rok'] = pd.to_datetime(df_orders.Data).dt.year
df_month_orders = df_month_orders.groupby(
    ['rok', "miesiac", 'name'], as_index=False).sum()
cols = ['rok', 'miesiac', 'name', 'Wielkosc_zamowienia']
df_month_orders = df_month_orders[cols]
df_month_orders
df_month_orders_1 = df_month_orders[['rok', 'miesiac', 'name']]


# new data frame with the compelete months of the four years for each product and with 0 in order column
years = [2019, 2020, 2021, 2022]
months = range(1, 13)
products_names = df_sale.name.to_list()
data = list(it.product(years, months, products_names))

month_orders_all = pd.DataFrame(
    data, columns=["rok", "miesiac", "name"])
month_orders_all = month_orders_all.sort_values(by=["rok", "miesiac", "name"])
month_orders_all = month_orders_all.reset_index(drop=True)
orders = [0] * 4800
month_orders_all['Wielkosc_zamowienia'] = orders
month_orders_all_1 = month_orders_all[['rok', 'miesiac', 'name']]

# this loop adds the order size to a new data frame that has
# complete months of the four years for each product as a result
# this gives a new data frame where the previously missing months
# for some products(those that were not ordered) have order sizes of 0
counter = 0
for i in range(len(month_orders_all)):
    a = month_orders_all_1.loc[i]
    b = df_month_orders_1.loc[counter]

    if a.equals(b):
        month_orders_all.loc[i] = df_month_orders.loc[counter]
        counter += 1

# Data for whole warehouse
year_orders_all = month_orders_all[['rok', 'miesiac', 'Wielkosc_zamowienia']]
year_orders_all = year_orders_all.groupby(
    ['rok', 'miesiac'], as_index=False).sum()

### Plots size

In [ ]:
plot_width = 7.5
plot_height = 2.6

#### Orders in time for whole warehouse BAR PLOT

In [ ]:
def create_ware_sale_plot(year_given):
    y = year_orders_all[(year_orders_all.rok == year_given)
                        ].Wielkosc_zamowienia
    x = year_orders_all[(year_orders_all.rok == year_given)].miesiac

    output = plt.figure()
    output.set_size_inches(plot_width+0.2, plot_height)
    plt.bar(x, y, color='lightblue')
    plt.xticks(x, fontsize=12)
    plt.yticks([0, 200, 400, 600, 800, 1000], fontsize=12)
    plt.tick_params(axis='x', length=0)
    plt.ylabel('Wielkość sprzedaży', fontsize=14)
    plt.xlabel(f'Miesiące {year_given} roku', fontsize=16)
    output.tight_layout()
    return output


warehouse_sale_plot = pn.interact(
    create_ware_sale_plot, year_given=year_select)
warehouse_sale_plot = warehouse_sale_plot[1]

In [ ]:
month_orders_all[(month_orders_all.Wielkosc_zamowienia > 15)
                 ].Wielkosc_zamowienia

#### Orders in time for specific product BAR PLOT

In [ ]:
def create_sale_month_plot(name_1, year_given):
    x = month_orders_all[(month_orders_all['name'] == name_1) &
                         (month_orders_all.rok == year_given)].miesiac
    y = month_orders_all[(month_orders_all['name'] == name_1) &
                         (month_orders_all.rok == year_given)].Wielkosc_zamowienia

    output = plt.figure()
    output.set_size_inches(plot_width, plot_height)
    # plt.bar(x, y, color='#de0173')
    plt.bar(x, y, color='lightblue')
    plt.ylabel('Wielkość sprzedaży', fontsize=14)
    plt.xticks(x, fontsize=12)
    plt.xlabel(f'Miesiące {year_given} roku', fontsize=16)
    plt.yticks([2, 4, 6, 8, 10, 12, 14, 16], fontsize=12)
    plt.tick_params(axis='x', length=0)
    output.tight_layout()
    return output


plot_orders_month = pn.interact(
    create_sale_month_plot, name_1=select, year_given=year_select)
plot_orders_month = plot_orders_month[1]

## Demand forecasting ML

In [ ]:
data_ML = df_orders.copy()
data_ML.index = pd.to_datetime(data_ML.Data)

# Rejecting unnecessary columns and filling in gaps with zeros
to_train_data_ML = data_ML.drop(
    columns=["EAN", "Calkowita_wartosc_zamowienia", "id_order"])
to_train_data_ML = to_train_data_ML.groupby(
    ["product_id", pd.Grouper(key='Data', freq='MS')])['Wielkosc_zamowienia'].sum()
to_train_data_ML = (to_train_data_ML.reset_index(level=0).groupby('product_id')[
    'Wielkosc_zamowienia'].apply(lambda x: x.asfreq('MS')).reset_index())
to_train_data_ML = to_train_data_ML.fillna(0)

max_prod_id = max(to_train_data_ML["product_id"])

### Functions to get values for all months for products

In [ ]:
def get_values(df, id_num):
    # get values for all months for one product
    g = df[df["product_id"] == id_num].values
    return g[:, 2]


def get_all_values(df):
    # get values for all months for all the products
    l = []
    for i in range(1, max_prod_id):
        l.append(get_values(df, i)[:46])
    return l

In [ ]:
get_values(df_orders, 100)

In [ ]:
# get all the data in one array
all_data = np.array(get_all_values(to_train_data_ML))

### Training data preparation

In [ ]:
# Training data preparation -- we use the window of 12 so
# we input 12 months of sales to the neural network and
# we predict only one point into the future. We scrap the
# data to get all possible pairs (12,1) -- 12 input points
# and 1 output point

window = 12

x_data = []
y_data = []

for x in all_data:
    for i in range(0, 46-window):
        x_data.append(x[i:i+window])
        y_data.append(x[i+window])

# we split the data to train and validation (80% and 20%)
# it is standard procedure
l = int(len(x_data) * 0.8)

x_train = np.array(x_data)[:l]
y_train = np.array(y_data)[:l]

x_val = np.array(x_data)[l:]
y_val = np.array(y_data)[l:]

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
# x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))


x_train = np.asarray(x_train).astype('float32')
x_val = np.asarray(x_val).astype('float32')

In [ ]:
# hyperparameters setup
epochs = 200
batch = 128
lr = 0.0001
adam = tf.keras.optimizers.Adamax()

# model setup (it is the same as for jewelery sale predition)
model_lstm = Sequential()
model_lstm.add(LSTM(165, input_shape=(x_train.shape[1], x_train.shape[2])))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(56))
model_lstm.add(Dropout(0.5))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer=adam)
model_lstm.summary()

### LSTM model training

In [ ]:
# training
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

lstm_history = model_lstm.fit(x_train, y_train, validation_data=(
    x_val, y_val), epochs=epochs, verbose=2, callbacks=[callback])

In [ ]:
# plot the loss curves -- just the check if the
# training went all good
plt.plot(lstm_history.history['loss'], label="Dane treningowe")
plt.plot(lstm_history.history['val_loss'], label="Dane testowe")
plt.xlabel("Epoka")
plt.ylabel("MAE")

plt.legend()

### Model prediction

In [ ]:
def predict(model, seq):
    his = []
    seq = np.reshape(seq[-12:], (1, -1))
    for i in range(12):
        a = model(seq)
        his.append(a[0][0].numpy())
        seq = np.hstack((seq, a.numpy()))[:, 1:]
        seq = tf.stack(seq)

    return his

### Model validation

In [ ]:
my_test = all_data[:, :12].astype(int)
my_test = pd.DataFrame(my_test)

my_valid = all_data[:, 12:24]
my_valid = pd.DataFrame(my_valid)

In [ ]:
model_reg = linear_model.LinearRegression().fit(my_test, my_valid)
prediction_regression = model_reg.predict(my_test)

In [ ]:
def create_forecast_plot(name_1, year_given):
    y = month_orders_all[(month_orders_all['name'] == name_1) &
                         (month_orders_all.rok == year_given)].Wielkosc_zamowienia

    output = plt.figure()
    output.set_size_inches(plot_width, plot_height+0.7)
    plt.bar(range(1, 13), predict(model_lstm, y), color='#de0173')
    plt.ylabel('Wielkość sprzedaży', fontsize=14)
    plt.xticks(range(1, 13), fontsize=12)
    plt.xlabel(f'Prognoza na {year_given+1} rok', fontsize=16)
    plt.yticks([2, 4, 6, 8, 10, 12, 14, 16], fontsize=12)
    plt.tick_params(axis='x', length=0)
    output.tight_layout()
    return output


plot_orders_forecast_months = pn.interact(
    create_forecast_plot, name_1=select, year_given=year_select)
plot_orders_forecast_months = plot_orders_forecast_months[1]

### Forecast for whole warehouse

In [ ]:
def create_forecast_plot_warehouse(year_given):
    x = year_orders_all[(year_orders_all.rok == year_given)].miesiac
    y = year_orders_all[(year_orders_all.rok ==
                         year_given)].Wielkosc_zamowienia

    output = plt.figure()

    plt.bar(x, predict(model_lstm, y), color='#de0173')
    plt.ylabel('Wielkość sprzedaży', fontsize=14)
    plt.xticks(range(1, 13), fontsize=12)
    plt.xlabel(f'Prognoza na {year_given+1} rok', fontsize=16)
    plt.yticks(fontsize=12)
    plt.tick_params(axis='x', length=0)
    output.tight_layout()
    output.set_size_inches(plot_width, plot_height+0.7)
    return output


plot_orders_forecast_warehouse = pn.interact(
    create_forecast_plot_warehouse, year_given=year_select)
plot_orders_forecast_warehouse = plot_orders_forecast_warehouse[1]
plot_orders_forecast_warehouse

In [ ]:
html_table_mass = pn.pane.HTML("""
<table>
<thead>
  <tr style="background-color:lightblue;">
    <th colspan="3">PRODUKT MASOWY</th>
  </tr>
</thead>
<tbody>
  <tr style="background-color:lightgrey;">
    <td>Ilość sztuk zapasu</td>
    <td>75-100 </td>
    
  </tr>
  <tr style="background-color:lightblue;">
    <td>Wystarczalności <br>magazynowej</td>
    <td>150-300 dni</td>
    
  </tr>
  <tr style="background-color:lightgrey;">
    <td>Rotacji <br>magazynowej</td>
    <td>2< </td>
    
  </tr>
  <tr style="background-color:lightblue;">
    <td>Obrotu magazynowego</td>
    <td>4-10%</td>
    
  </tr>
  <tr style="background-color:lightgrey;">
    <td>Marża</td>
    <td>20-25%</td>
    
  </tr>
  <tr style="background-color:lightblue;">
    <td>Udział w sprzedaży</td>
    <td>5-10%</td>
   
</tbody>
</table>
""")

In [ ]:
html_table_seasonal = pn.pane.HTML("""
<table>
<thead>
  <tr style="background-color:lightblue;">
    <th colspan="3">PRODUKT SEZONOWY</th>
  </tr>
</thead>
<tbody>
  <tr style="background-color:lightgrey;">
    <td>Ilość sztuk zapasu</td>
    <td>5-10 </td>
  </tr>
  
  <tr style="background-color:lightblue;">
    <td>Wystarczalności <br>magazynowej</td>
    <td>50-75 dni</td>
    
  </tr>
  <tr style="background-color:lightgrey;">
    <td>Rotacji <br>magazynowej</td>
    <td>1-2</td>
    
  </tr>
  <tr style="background-color:lightblue;">
    <td>Obrotu magazynowego</td>
    <td>4-10%</td>
    
  </tr>
  <tr style="background-color:lightgrey;">
    <td>Marża</td>
    <td>25-30%</td>
    
  </tr>
  <tr style="background-color:lightblue;">
    <td>Udział w sprzedaży</td>
    <td>1-5%</td>
    
  </tr>
</tbody>
</table>
""")

In [ ]:
html_table_luxury = pn.pane.HTML("""
<table>
<thead>
  <tr style="background-color:lightblue;">
    <th colspan="3">PRODUKT LUKSUSOWY</th>
  </tr>
  
</thead>
<tbody>
  <tr style="background-color:lightgrey;">
    <td>Ilość sztuk zapasu</td>
    <td>5-10 </td>
  </tr>

  <tr style="background-color:lightblue;">
    <td>Wystarczalności <br>magazynowej</td>
    <td>20-30 dni</td>
  </tr>

  <tr style="background-color:lightgrey;">
    <td>Rotacji <br>magazynowej</td>
    <td> <1 </td>
  </tr>

  <tr style="background-color:lightblue;">
    <td>Obrotu magazynowego</td>
    <td>1-2%</td>
    
  </tr>
  <tr style="background-color:lightgrey;">
    <td>Marża</td>
    <td>30-35%</td>
    
  </tr>
  <tr style="background-color:lightblue;">
    <td>Udział w sprzedaży</td>
    <td><1%</td>
   
</tbody>
</table>
""")

## 5. Composing Dashboard

In [ ]:
controls_width = 250
controls_height = 300

controls = pn.Column(pn.pane.PNG("Prestashop-Square.png", sizing_mode='scale_both'), pn.pane.Markdown("## Kontrolery"),
                     year_select, select, pn.pane.Markdown("### Typ produktu"), pn.Tabs(('Masowy', html_table_mass),
                                                                                        ('Luksusowy', html_table_luxury), (
                                                                                            'Sezonowy', html_table_seasonal)
                                                                                        ), width=controls_width, )

widgets_width = 588
widgets_height = 260

plot_warehouse_sale = pn.Column('## Sprzedaż historyczna wszystkich pozycji asortymentowych', warehouse_sale_plot,
                                width=widgets_width, height=widgets_height)

plot_sale_month = pn.Column('## Sprzedaż historyczna produktu', plot_orders_month,
                            width=widgets_width, height=widgets_height)

plot_forecast_product = pn.Column('## Prognoza popytu wybranego produktu', plot_orders_forecast_months,
                                  width=widgets_width, height=widgets_height)
plot_forecast_warehouse = pn.Column('## Prognoza popytu całości asortymentu', plot_orders_forecast_warehouse,
                                    width=widgets_width, height=widgets_height)
KPIs_sale_warehouse = pn.Column('## Wskaźniki sprzedaży sklepu',
                                pn.Row(pn.Column(number_total_income, number_total_profit),
                                       pn.Column(number_mean_total_markup, number_mean_total_margin)), width=widgets_width, height=widgets_height)
KPIs_logistic_warehouse = pn.Column('## Wskaźniki logistyczne sklepu',
                                    pn.Row(pn.Column(number_Total_warehouse_value, number_warehouse_sufficiency_rate),
                                           pn.Column(number_warehouse_turnover_rate, number_mean_rotation)), width=widgets_width, height=widgets_height)

KPIs_sale = pn.Column('## Wskaźniki sprzedaży produktu',
                      pn.Row(pn.Column(number_product_income, number_profit),
                             pn.Column(number_sale_share, number_margin)), width=widgets_width, height=widgets_height)

KPIs_logistic = pn.Column('## Wskaźniki logistyczne produktu',
                          pn.Row(pn.Column(pn.Row(number_stock_value, number_product_stock, height=120), number_product_sufficiency_rate),
                                 pn.Column(pn.Row(number_turnover_rate, height=120), pn.Row(number_product_rotation))), width=widgets_width, height=widgets_height)


template1 = pn.template.MaterialTemplate(
    title='Narzędzie analityczne do zarządzania magazynem dla sklepów internetowych',
    sidebar=[controls],
    main=[header_title_ean,
          pn.Row(pn.Tabs(('WYBRANA POZYCJA', KPIs_sale),
                         ('CAŁY ASORTYMENT', KPIs_sale_warehouse), width=widgets_width),
                 pn.Tabs(('WYBRANA POZYCJA', plot_sale_month),
                         ('CAŁY ASORTYMENT', plot_warehouse_sale))

                 ),
          pn.Row(pn.Tabs(('WYBRANA POZYCJA', KPIs_logistic),
                         ('CAŁY ASORTYMENT', KPIs_logistic_warehouse), width=widgets_width),
                 pn.Tabs(('WYBRANA POZYCJA', plot_forecast_product)
                         )
                 )

          ],

    accent_base_color="lightblue",
    header_background="lightblue",
    sidebar_width=275,
)

template1.show()
template1.save("tabelka")